# Filters performance logs out of an Android log file
Log file aquired by `adb logcat  -T "08-22 13:50:00.000" -v time,printable measure:I *:S > "2023-08-21-gp6.txt"`

https://developer.android.com/tools/logcat


In [13]:
from datetime import date

# datetime object containing current date and time
today = date.today()

experiment = "scroll"

input_file_name = "raw/2023-11-30-scroll.txt"
output_file_name = "csv/{}_{}.csv".format(today, experiment)
trace_keyword = "I/measure" # for measuring benchmarks
headers = "name,time"

In [14]:
import re

# open the input file for reading
with open(input_file_name, 'r') as input_file:
    # read in all lines from the input file
    lines = input_file.readlines()


# cut off everything before measure and the process id:, including the word itself
def convert_input_to_output(input_string):
    pattern = rf'.*(\): )'
    match = re.search(pattern, input_string)
    if match:
        return match.group(0) + '\n'
    return None

def extract_output(input_string):
    index = input_string.find('): ')
    if index != -1:
        return input_string[index + 3:]
    return None

# https://stackoverflow.com/a/59082116
def replace_last_occurrence(input_string):
    return ','.join(input_string.rsplit(':', 1))

# filter input to just contain lines with the "measure: " keyword
filtered_lines = [line for line in lines if trace_keyword in line]

# cut off everything before measure:, including the word itself
formatted_lines = [extract_output(line) for line in filtered_lines]

csv_lines = [replace_last_occurrence(line) for line in formatted_lines]

# open the output file for writing
with open(output_file_name, 'w') as output_file:
    # write out the filtered lines to the output file
    output_file.write(headers + "\n")
    output_file.writelines(csv_lines)
